<a href="https://colab.research.google.com/github/s0ye0nyang/News-Recomendation-System/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

fatal: destination path 'khaiii' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: 02ccb06 | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunte

In [ ]:
!pip install kss

In [ ]:
import gensim
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
# from konlpy.tag import Okt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ++

## 동사 원형

In [ ]:
from khaiii import KhaiiiApi
import re
import kss

api = KhaiiiApi()
data = pd.read_csv('NaverNews.csv',names=['a','b'], encoding='utf-8')
lines = data['a'].values
# print(lines)

tokenized_text = []
for i, line in enumerate(lines):
    if i > 100:    
        break
    line = line.strip()
    for sent in kss.split_sentences(line):
        tokenized_text.append(sent.strip())

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }


def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

cleaned_corpus = []
for sent in tokenized_text:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [ ]:
for i in range(0, 10):
    print(cleaned_corpus[i])

기사
/    /   flash 오류를 우회하기 위한 함수 추가function   -  flash  -  removeCallback (  )   {  }   [ 앵커 ]  보신대로 윤석열 검찰총장에 대한 징계위원회가 오늘 오전부터 9시간 넘게 열렸습니다만 ,  결론은 내놓지 못한 채 일단 끝났습니다 .
최영일 시사평론가와 함께 향후 징계위원회 전망을 보다 자세히 살펴보겠습니다 .
평론가님 ,  원래 징계위원회가 이렇게까지 오래 진행되는 겁니까 ?
왜 오늘 결론내지 못한 겁니까 ?
[ 앵커 ]  가장 큰 쟁점은 이른 바   '  판사 사찰 문건  '   의혹이죠 .
그간 법무부와 윤 총장 측의 입장이 팽팽하게 맞서기도 했는데요 .
오늘 징계위에서도 일단 양 쪽의 입장을 듣고 끝냈다 ,
이렇게 봐야겠죠 ?
[ 앵커 ]  그런데 윤석열 총장 측은 그동안 계속 감찰과 징계 절차가 부당했다 ,


In [ ]:
def clean_text(texts):
    corpus = []
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    for i in range(0, len(texts)):
        result = hangul.sub('', str(texts[i]))
        corpus.append(result)
    return corpus

basic_preprocessed_corpus = clean_text(cleaned_corpus)
print(basic_preprocessed_corpus)

significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

pos_tagged_corpus = pos_text(basic_preprocessed_corpus)

['기사', '        오류를 우회하기 위한 함수 추가                     앵커   보신대로 윤석열 검찰총장에 대한 징계위원회가 오늘 오전부터 시간 넘게 열렸습니다만   결론은 내놓지 못한 채 일단 끝났습니다 ', '최영일 시사평론가와 함께 향후 징계위원회 전망을 보다 자세히 살펴보겠습니다 ', '평론가님   원래 징계위원회가 이렇게까지 오래 진행되는 겁니까 ', '왜 오늘 결론내지 못한 겁니까 ', ' 앵커   가장 큰 쟁점은 이른 바     판사 사찰 문건     의혹이죠 ', '그간 법무부와 윤 총장 측의 입장이 팽팽하게 맞서기도 했는데요 ', '오늘 징계위에서도 일단 양 쪽의 입장을 듣고 끝냈다 ', '이렇게 봐야겠죠 ', ' 앵커   그런데 윤석열 총장 측은 그동안 계속 감찰과 징계 절차가 부당했다 ', '이렇게 주장했었죠 ', '그래서 오늘 징계위도 나오지 않았던 겁니까 ', ' 앵커   앞서 보셨지만   윤 총장 측은 오늘 징계위가 시작되자마자 징계위원들에 대한 기피 신청을 진행했습니다 ', '어떤 점이 문제가 됐던 겁니까 ', ' 앵커    만일 윤석열 총장 측에서 절차적 정당성을 이유로 법적인 대응을 진행한다면 징계위원들의 구성이나 기피 신청 절차를 다시 한 번 따져볼 수도 있는 겁니까 ', ' 앵커   어쨌든 윤 총장은 향후 징계위 결정에 따라 어떻게 대응할 지를 결정할텐데요 ', '징계위원회가 일에 재개되지 않습니까   일엔 어떤 점을 살펴보는 겁니까 ', ' 앵커   그러면 일에는 징계위의 결론이 나오는 겁니까 ', ' 앵커   일이든   일 이후든   징계위가 내릴 징계 수위에 따라 그 파장도 만만치 않을 것 같은데요 ', ' 앵커   앞서 말씀드린대로   윤 총장 측이 법적인 대응까지 거론하고 있는 만큼   이 문제가 완전히 마무리되기까지도 꽤 오랜 시간이 걸릴 것 같습니다 ', '     코로나 팩트체크     제대로 알아야 이긴다 기후는 말한다  생존을 위한 전환     탄소중립     더 빠르고 정확한 소식을

In [ ]:
for i in range(0, 100):
    print(pos_tagged_corpus[i])

In [ ]:
# 동사원형을 찾는 규칙 정의
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.') # 동사
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX') # 명사
p3 = re.compile('[가-힣A-Za-z0-9]+/VV') # 형용사
p4 = re.compile('[가-힣A-Za-z0-9]+/VX') # 부사

def stemming_text(text):
    corpus = []
    for sent in text:
        # if sent in exception word:
        #   corpus.append(sent)
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        ori_list = ori_sent.split(' ')
        temp = [[ori] for ori in ori_list]
        print(temp[0])
        corpus.extend(temp)
        # print("orisent:",corpus,'\n')
    return corpus

In [ ]:
stemming_corpus = stemming_text(pos_tagged_corpus)

['기사/NNG']
['오류/NNG']
['최영일/NNP']
['평론가/NNG']
['왜/MAG']
['앵커/NNG']
['그간/NNG']
['오늘/NNG']
['이렇/VA']
['앵커/NNG']
['이렇/VA']
['그래서/MAJ']
['앵커/NNG']
['점/NNG']
['앵커/NNG']
['앵커/NNG']
['징계/NNG']
['앵커/NNG']
['앵커/NNG']
['앵커/NNG']
['코로/NNG']
['오류/NNG']
['김근식/NNP']
['윤석열/NNP']
['김/NNP']
['오늘/NNG']
['국회/NNG']
['개정안/NNG']
['문재/NNP']
['김승현/NNP']
['조선일보/NNP']
['오류/NNG']
['이낙연/NNP']
['연합/NNG']
['클릭/NNG']
['오류/NNG']
['공동/NNG']
['의원/NNG']
['경찰/NNG']
['필리버스터/NNP']
['의원/NNG']
['이어/MAG']
['또/MAG']
['서울/NNP']
['공동/NNG']
['의원/NNG']
['경찰법/NNG']
['아/NNP']
['아울러/MAG']
['의원/NNG']
['서울/NNP']
['공동/NNG']
['장/NNP']
['김상희/NNP']
['의원/NNG']
['국정원/NNG']
['공감언론/NNG']
['오류/NNG']
['이철규/NNP']
['사진/NNG']
['국민/NNG']
['의원/NNG']
['북한/NNP']
['아울러/MAG']
['의원/NNG']
['한편/MAG']
['권오석/NNG']
['하/XSA']
['오류/NNG']
['일/NNG']
['앞서/MAG']
['대하다/VV']
['같/VA']
['데일리안/NNP']
['오류/NNG']
['뉴스/NNG']
['이날/NNG']
['경찰/NNG']
['의원/NNG']
['의원/NNG']
['의원/NNG']
['의원/NNG']
['윤석열/NNP']
['의원/NNG']
['의원/NNG']
['의원/NNG']
['이버/NNP']
['오류/NNG']
['뉴스/NNG']
['거대/NNG

In [ ]:
import string
#stopwords = ['조선일보/NNP','이/NNG','사진제공/NNG','데/NNB', '좀/MAG', '수/NNB', '등/NNB','이제/MAG','그러다/VV','그리고/MAJ','앵커/NNG', '또/MAG', '말/NNG','하지만/MAJ','이어/MAG', '그러나/MAJ','곳/NNG','앞/NNG', '특히/MAG','반면/NNG','최고/NNG','같/VA' ,'최/NNP'] # 추가할 단어 넣기
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')

# 불용어 데이터를 읽어와 불용어들을 제거
# string.puctuation을 불용어에 추가하여 특수문자도 제거!
def define_stopwords(path):
  SW = set()

  for i in string.punctuation:
    SW.add(i)

  with open(path, encoding='utf-8') as f:
    for word in f:
      SW.add(word)

  return SW



def remove_stopword_text(text):
    corpus = []
    for word in text:
        word = word[0]
        if word not in SW:
            word = hangul.sub('', word)
            corpus.append([word])
    return corpus

removed_stopword_corpus = remove_stopword_text(stemming_corpus)

print(removed_stopword_corpus)

[['기사'], ['오류'], ['우회하다'], ['위하다'], ['함수'], ['추가'], ['보'], ['신'], ['윤석열'], ['검찰'], ['총장'], ['대하다'], ['징계'], ['위원회'], ['오늘'], ['오전'], ['시간'], ['넘다'], ['열리다'], ['결론'], ['내놓다'], ['못하'], ['채'], ['일단'], ['끝나다'], ['최영일'], ['시사'], ['평론가'], ['함께'], ['향후'], ['징계'], ['위원회'], ['전망'], ['보다'], ['자세히'], ['살펴보다'], ['평론가'], ['원래'], ['징계'], ['위원회'], ['이렇'], ['오래'], ['진행되다'], ['겁다'], ['왜'], ['오늘'], ['결론'], ['내'], ['못하'], ['겁'], ['가장'], ['크'], ['쟁점'], ['이르'], ['판사'], ['사찰'], ['문건'], ['의혹'], ['그간'], ['법무부'], ['윤'], ['총장'], ['측'], ['입장하다'], ['맞서다'], ['하'], ['오늘'], ['징계위'], ['일단'], ['양'], ['쪽'], ['입장'], ['듣다'], ['끝내다'], ['이렇'], ['보다'], ['하'], ['그런데'], ['윤석열'], ['총장'], ['측'], ['그동안'], ['계속'], ['감찰'], ['징계'], ['절차'], ['부당하다'], ['이렇'], ['주장하다'], ['그래서'], ['오늘'], ['징계위'], ['나오다'], ['않'], ['겁'], ['앞다'], ['서'], ['보다'], ['윤'], ['총장'], ['측'], ['오늘'], ['징계위'], ['시작되다'], ['징계'], ['위원'], ['대하다'], ['기피'], ['신청'], ['진행하다'], ['점'], ['문제'], ['되다'], ['겁'], ['만일'], ['윤석열'], ['총장'], ['측'], ['절차'], ['이유'], ['법'], ['대응'], ['진행

In [ ]:
from gensim.models import Word2Vec

# word_list = ' '.join(removed_stopword_corpus).split(' ')

model = Word2Vec(removed_stopword_corpus,         # 리스트 형태의 데이터
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=100,     # 벡터 크기
                 window=3,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=3,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=2)

model.save('word2vec.model')

In [ ]:
model.wv.most_similar('시작되다')

# model.wv.similarity('제공','내일')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.03563786